# Load and initialize

settings

In [ ]:
# sample number

j='1'
group_ind = 0
# n_samples = 10000
    # create visualization (umap, dbscan are always shown)
visualize = True
plotUMAP  = True
    # show figures in notebook
show = False

saveSVG = False

dir_data = 'Data_04042023'




envirement

In [ ]:
import os

import time 
start = time.process_time()
import datetime 
date = datetime.date.today().strftime("%d%m%Y")
import pathlib
import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics

# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters


parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dir_data = os.path.abspath(os.path.join(parent_dir, os.pardir))+f'/{dir_data}/'
sys.path.append(parent_dir+'/functions/')


from usefull_functions import *
from functions import *
from plot_functions import *
from clustering_functions import *
# from epinuc import *



%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2





TF_ENABLE_ONEDNN_OPTS=0

load data; normalized (CD45-)  

In [ ]:
j,group_ind = getJ(j,group_ind)
dir_plots = os.path.abspath(os.path.join(parent_dir, os.pardir))+f'/_sample_{j}_{date}/'
folderExists(dir_plots)
settings =        (dir_plots,show,saveSVG)

if 'b' not in j:
    singleSample = True
else:
    singleSample = False
print(f'singleSample is: {singleSample}')

# k_orig = pickle_load('k'+j+'_dict',dir_data )['k']
# k = k_orig.copy()
# CAll=pd.concat([k]).copy()

k = pickle_load('k'+j+'_dict',dir_data )['k']
CAll=pd.concat([k.copy()])
print(len(k))


dict = pickle_load('k'+j+'_names_dict',dir_data )
NamesAll = dict['NamesAll']
CellIden = dict['CellIden']
EpiCols = dict['EpiCols']
Core = dict['Core']
CellIden_EpiCols = CellIden + EpiCols
dict = None

groups = [['CellIden','EpiCols','CellIden+EpiCols'],[CellIden,EpiCols,CellIden_EpiCols]]

names = [[f'samp{j}_CellIden_',f'samp{j} CellIden Based'],
         [f'samp{j}_EpiCols_',f'samp{j} EpiCols Based'],
         [f'samp{j}_CellIden+EpiCols_',f'samp{j} CellIden+EpiCols Based']]

params = ['iden','epi','all']

Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

In [ ]:
# if show:

#     for features in [NamesAll,Core,CellIden,EpiCols,CellIden_EpiCols]:
#         print(len(features))
#         print(features)


# Cell Identity

In [ ]:

figname,title = names[group_ind]
name,featureGroup = groups[0][group_ind],groups[1][group_ind]
print(name,title,figname)


### UMAP

plot for each feature according to its intensity

In [ ]:
min_dist,n_neighbors = getValsCsv(dir_data,vars = [params[group_ind],'umap',j])
print(min_dist,n_neighbors)

umapData = calculate_umap(CAll[featureGroup],n_neighbors, min_dist)
  
drawUMAP(umapData, ['H4'],CAll,settings,
            title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
            Figname = '1_'+figname+'UMAP_(core)')

### Dbscan and clusters

In [ ]:

eps,min_samples = getValsCsv(dir_data,vars = [params[group_ind],'db',j])
print(eps,min_samples)

dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps,min_samples)

colors = drawDbscan(dbscanData,labels,core_samples_mask,settings,
        title=f'min_samples = {min_samples}, eps = {eps}',
        figname = '1_'+figname+'dbscan')

In [ ]:
plotClusters(k,umapData,labels,colors,NamesAll,settings,
             title = 'Clusters ' + title,
             figname = '1_'+figname+'Clusters')
             



### HeatMap by clusters

In [ ]:
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'HeatMap ' + title + ': '+group_name,
                figname = '1_'+figname+'ClustHeatMap_by_'+group_name)
    

### HeatMap by sample

In [ ]:
if visualize and not singleSample :
    group_name,group = groups[0][-1],groups[1][-1]
    HeatMap(k[k.Clust!=-1],group,settings,clustFeature='by_sample',
            title = 'HeatMap ' + title + ': '+ group_name+' by_sample',
            figname = '1_'+figname+'SampHeatMap_'+group_name)
        


### MeanDist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title = 'MeanDist ' + title + ': '+group_name,
                figname = '1_'+figname+'ClustMeanDist_by_'+group_name)


## sample percentage

In [ ]:
if visualize and not singleSample:
    # unique by_sample values
    sampNums = np.unique(k['by_sample'])
    for sampNum in sampNums:
        drawUMAPbySample(umapData, k, sampNum,'by_sample',settings,
                            title =title+' UMAP', 
                            Figname = '4_'+figname+f'UMAP_sample{sampNum}_')

    arr =[]
    clustNums = np.unique(k.Clust); clustNums = clustNums[clustNums!=-1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum
        PercentageBySample = ClustPercentageBySample(k[ind],'by_sample')

        arr.append([clustNum,PercentageBySample])
    saveCsv(dir_plots,figname,arr)

## features on UMAP

In [ ]:
if visualize and plotUMAP:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title = 'UMAP ' + title, 
            Figname = '2_'+figname+'UMAP_')



## scatter figures

In [ ]:
if visualize:
    try:
        scatter(k,'NCad','ECad',title,figname,settings)
    except:
        print('no ncad in sample')

## clear data

In [ ]:
# dbscanData, umapData,k, CAll,labels = None,None,None,None,None
# imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])